In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#NER
각 단어가 어떠한 개체명(장소, 시간, 등)을 가르키는지를 확인하는 task  
dataset:https://www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset

##Data PreProcessing  
제공된 data는 이미 tokenization이 되어있는 형태로 제공된다.  
그렇기에 vocab dictionary를 직접 구성하고 각 tokenization된 token들을 indexing하여 숫자형태의 데이터로 변환환다.  
이때 class에 index를 부여해주는 scikit learn의 LabelEncoder를 사용해서 변환환다.  
또한 모든 token의 수를일정하게 맞춰주어야만 tensor 연산이 가능하기 때문에 <PAD>에 대한 indexing도 추가한다.


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data_df = pd.read_csv("/content/drive/MyDrive/여름 NLP/5일차/dataset/NER dataset.csv", encoding="ISO-8859-1")
data_df =data_df.loc[:100000,:]
print(data_df)

dummlabel = ["A","B","C","D"]
le = LabelEncoder()
le.fit(dummlabel)
print(le.transform(dummlabel))

token_encoder = LabelEncoder()
token_encoder.fit(["<PAD>"] + list(data_df.loc[:,"Word"]))

data_df.loc[:,"Word"] = token_encoder.transform(data_df.loc[:,"Word"])
token_pad_index = token_encoder.transform(["<PAD>"])
print(data_df)

label_encoder = LabelEncoder()
label_encoder.fit(["<PAD>"]+list(data_df.loc[:,"Tag"]))
label_pad_index = label_encoder.transform(["<PAD>"])

data_df.loc[:,"Tag"] = label_encoder.transform(data_df.loc[:,"Tag"])
print(data_df)

data_list = []
label_list = []
for i in range(100000):
  if type(data_df.loc[i,"Sentence #"]) is str:
    line = []
    labels = []
  line.append(data_df.loc[i,"Word"])
  labels.append(data_df.loc[i,"Tag"])

  if type(data_df.loc[i+1,"Sentence #"]) is str:
    data_list.append(line)
    label_list.append(labels)

print(len(data_list))


            Sentence #           Word  POS Tag
0          Sentence: 1      Thousands  NNS   O
1                  NaN             of   IN   O
2                  NaN  demonstrators  NNS   O
3                  NaN           have  VBP   O
4                  NaN        marched  VBN   O
...                ...            ...  ...  ..
99996              NaN      seriously   RB   O
99997              NaN              .    .   O
99998   Sentence: 4544  Demonstrators  NNS   O
99999              NaN       chanting  VBG   O
100000             NaN              "   ``   O

[100001 rows x 4 columns]
[0 1 2 3]
            Sentence #  Word  POS Tag
0          Sentence: 1  3536  NNS   O
1                  NaN  8074   IN   O
2                  NaN  5605  NNS   O
3                  NaN  6826  VBP   O
4                  NaN  7652  VBN   O
...                ...   ...  ...  ..
99996              NaN  9522   RB   O
99997              NaN    15    .   O
99998   Sentence: 4544  1295  NNS   O
99999              

##Dataset  
전체 데이터중 4000개의 data는 train 나머지는 test로 사용한다.  
사전에 tokenization과 indexing된 데이터의 크기를 맞추기위해 data에 padding을 추가한다.  
  padding의 value는 <PAD> token의 index를 사용한다.

In [4]:
from torch.utils.data import Dataset, DataLoader
import torch

train_data_list = data_list[:4000]
test_data_list = data_list[4000:]
train_label_list = data_list[:4000]
test_label_list = data_list[4000:]

class MyDataset(Dataset):
  def __init__(self,data_list,label_list) -> None:
      super().__init__()
      self.data_list = data_list
      self.label_list = label_list

  def __len__(self):
    return len(self.data_list)

  def __getitem__(self,index):
    data_pad = torch.zeros(50,dtype=torch.int64) + token_pad_index
    label_pad = torch.zeros(50) + label_pad_index
    data = torch.cat([torch.IntTensor(self.data_list[index]),data_pad])[:50]
    label = torch.cat([torch.IntTensor(self.label_list[index]),label_pad])[:50]

    return data.flip(-1), label.flip(-1)

train_dataset = MyDataset(train_data_list,train_label_list)
test_dataset = MyDataset(test_data_list,test_label_list)

for i in train_dataset:
  print(i)
  break

batch_size = 3
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

for i in train_dataloader:
  print(i[0].shape)
  print(i[1].shape)
  break


(tensor([  487,   487,   487,   487,   487,   487,   487,   487,   487,   487,
          487,   487,   487,   487,   487,   487,   487,   487,   487,   487,
          487,   487,   487,   487,   487,   487,    15,  5371, 10223,  6574,
        10418,   969,  8074, 10844, 10225,  5586,  4193,  1958,  7062, 10705,
        10225,  8732, 10296,  2308, 10271,  7652,  6826,  5605,  8074,  3536]), tensor([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,    15.,
         5371., 10223.,  6574., 10418.,   969.,  8074., 10844., 10225.,  5586.,
         4193.,  1958.,  7062., 10705., 10225.,  8732., 10296.,  2308., 10271.,
         7652.,  6826.,  5605.,  8074.,  3536.], dtype=torch.float64))
torch.Size([3, 50])
torch.Size([3, 50])


##Model  
lstm을 위한 model과 seq2seq를 이용한 모델 두가지 구성  
seq2seq는 encoder model과 decoder model을 따로 구성하고 encoder model의 h,c를 decoer model의 입력으로 사용한다.  
이후 decoder model의 hidden state 들을 예측을 위해 사용한다.

In [9]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.emb = nn.Embedding(35179, 128)
      self.lstm = nn.LSTM(128,128, batch_first=True)
      self.ln1 = nn.Linear(128,20)
      self.relu = nn.ReLU()
  def forward(self,x):
    x = self.emb(x)
    x, (h,c) = self.lstm(x)
    print(x.shape)
    x = self.ln1(x)
    x = self.relu(x)

    return x


model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([3, 50])
torch.Size([3, 50, 128])
torch.Size([3, 50, 20])


In [8]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()

      self.emb = nn.Embedding(35179, 128)
      self.lstm = nn.LSTM(128,128, batch_first=True, bidirectional=True)
      self.ln1 = nn.Linear(128,20)

  def forward(self, x):
      x = self.emb(x)
      x, (h,c) = self.lstm(x)
      print(x.shape)
      x = self.ln1(x)

      return x

model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([3, 50])
torch.Size([3, 50, 256])


RuntimeError: ignored

In [23]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()

      self.en_emb = nn.Embedding(50000, 128)
      self.en_lstm = nn.LSTM(128,128, batch_first=True)
      self.en_ln1 = nn.Linear(128,20)

      
      self.de_emb = nn.Embedding(50000, 128)
      self.de_lstm = nn.LSTM(128,128, batch_first=True)
      self.de_ln1 = nn.Linear(128,20)

  def forward(self, x):
      x = self.en_emb(x)
      x, (h,c) = self.en_lstm(x)

      x, _ = self.de_lstm(x, (h,c))
      x = self.de_ln1(x)

      return x

model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([3, 50])
torch.Size([3, 50, 20])


optimization

In [25]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = myModel()
model.cuda()

#학습을 위한 optimizer와 loss function 설정
optimizer = Adam(model.parameters(), lr=0.0001)
lf = CrossEntropyLoss().cuda()


#100번의 에폭을 실행
for e in range(100):
  print("\n\nepoch ", e)
  epoch_loss = 0
  train_correct = 0 
  
  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in train_dataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()
    data = i[0]
    data = data.cuda()
    target = i[1]
    target = target.reshape(-1).cuda()
    #print(target.shape)

    #결과 도출 및 정답수 연산
    output = model(data).reshape(-1,20)
    #print(output.shape)
    pred_label = torch.argmax(output, dim=-1)
    #print(pred_label.shape)
    #print(target.shape)
    train_correct += sum(pred_label == target)
    #print(sum(pred_label == target))
    #print(pred_label)
    #print(target)
    # print(data)

    #loss연산
    loss = lf(output, target.long())
    #print(loss)

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()
  
  print(train_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0
  test_correct = 0 

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in test_dataloader:
      test_data = i[0]
      test_target = i[1]
      test_data = test_data.cuda()
      test_target = test_target.reshape(-1).cuda()
      #print(test_target.shape)

      test_output = model(test_data).reshape(-1,20)
      #print(test_output.shape)

      tloss = lf(test_output, test_target.long())
      test_pred_label = torch.argmax(test_output, dim=-1)
      #print(test_pred_label.shape)
      test_correct += sum(test_pred_label == test_target)
      test_loss += tloss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))
    


RuntimeError: ignored